#### Tuning for RBF-PSO Modelling of QFE Air Pressure
Some code using the "03" notebook cell.

In [1]:
from pathlib import Path
import pandas as pd, numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np
import pyswarms as ps, warnings ; warnings.filterwarnings("ignore")

# ── EDIT ONLY THESE TWO LINES WHEN YOU CHANGE STATION ── #
STATION_ID = "97230"
DATA_FILE  = Path(f"/home/rzby/airpressure_ann/ann_pressure_prediction/data/processed/clean_{STATION_ID}.parquet")
# ──────────────────────────────────────────────────────── #

SEED = 42


In [2]:
def add_time_feats(df):
    ang_h = 2*np.pi*df.index.hour/24
    ang_d = 2*np.pi*df.index.dayofyear/365.25
    df["sin_hour"], df["cos_hour"] = np.sin(ang_h), np.cos(ang_h)
    df["sin_doy"],  df["cos_doy"]  = np.sin(ang_d), np.cos(ang_d)
    return df

def add_lags(df, cols, lags=[1,2,4]):
    for c in cols:
        for k in lags:
            df[f"{c}_lag{k}"] = df[c].shift(k)
    return df

def kge(y, yhat):
    r = np.corrcoef(y, yhat)[0,1]
    alpha = yhat.std()/y.std()
    beta  = yhat.mean()/y.mean()
    return 1-np.sqrt((r-1)**2+(alpha-1)**2+(beta-1)**2)

from sklearn.cluster import MiniBatchKMeans
from numpy.linalg import pinv
import numpy as np

class RBFNetwork:
    """
    Radial-Basis-Function Neural Network with:
      • K centres from (MiniBatch) K-means
      • Gaussian basis ϕ(||x−c||)  with shared gamma
      • Closed-form ridge regression for output weights
    """
    def __init__(self, n_centres=300, gamma=None,
                 ridge=1e-4, random_state=42, batch_size=1024):
        self.n_centres  = n_centres
        self.gamma      = gamma        # if None compute from centres spread
        self.ridge      = ridge
        self.rs         = random_state
        self.batch_size = batch_size   # for MiniBatchKMeans

    # ───────────────── helpers ──────────────────
    @staticmethod
    def _euclid(a,b):
        return np.linalg.norm(a[:,None,:] - b[None,:,:], axis=2)

    def _rbf(self, X):
        d = self._euclid(X, self.C_)
        return np.exp(-(self.gamma * d**2))

    # ─────────────── public API ─────────────────
    def fit(self, X, y):
        # 1) choose K centres
        km = MiniBatchKMeans(n_clusters=self.n_centres,
                             random_state=self.rs,
                             batch_size=self.batch_size)
        km.fit(X)
        self.C_ = km.cluster_centers_

        # 2) set gamma if not user-supplied
        if self.gamma is None:
            d_max = np.max(self._euclid(self.C_, self.C_))
            sigma = d_max / np.sqrt(2*self.n_centres)
            self.gamma = 1 / (2*sigma**2)

        # 3) hidden matrix
        H = self._rbf(X)

        # 4) ridge regression: β = (HᵀH + λI)⁻¹ Hᵀ y
        I = self.ridge * np.eye(self.n_centres)
        self.B_ = pinv(H.T @ H + I) @ H.T @ y
        return self

    def predict(self, X):
        return self._rbf(X) @ self.B_
    
    def score(self, X, y):
        """Sklearn-compatible score = R-squared."""
        return r2_score(y, self.predict(X))



In [3]:
df = pd.read_parquet(DATA_FILE).set_index('index')
df.head()

,WMO_ID,RAINFALL_LAST_MM,TEMP_DEWPOINT_C_TDTDTD,TEMP_DRYBULB_C_TTTTTT,TEMP_WETBULB_C,WIND_SPEED_FF,RELATIVE_HUMIDITY_PC,PRESSURE_QFF_MB_DERIVED,PRESSURE_QFE_MB_DERIVED,month
index,,,,,,,,,,
2015-01-01 00:00:00+00:00,97230,-0.999845,0.342466,-0.085714,0.147059,0.176471,0.189448,1008.3,1007.8,-1.0
2015-01-01 03:00:00+00:00,97230,-1.000000,0.397260,0.257143,0.323529,0.411765,-0.165468,1008.0,1007.5,-1.0
2015-01-01 06:00:00+00:00,97230,-0.831008,0.150685,-0.580952,-0.205882,-0.764706,0.654676,1006.2,1005.7,-1.0
2015-01-01 09:00:00+00:00,97230,-0.996899,0.506849,0.200000,0.382353,0.411765,-0.016787,1005.4,1004.9,-1.0
2015-01-01 12:00:00+00:00,97230,-1.000000,0.205479,-0.085714,0.029412,-0.176471,0.055156,1008.0,1007.5,-1.0


In [4]:
# feature engineering
df = add_time_feats(df)
core = ["PRESSURE_QFF_MB_DERIVED","PRESSURE_QFE_MB_DERIVED","TEMP_DEWPOINT_C_TDTDTD","TEMP_DRYBULB_C_TTTTTT", "TEMP_WETBULB_C", "WIND_SPEED_FF","RAINFALL_LAST_MM", "RELATIVE_HUMIDITY_PC"]
df = add_lags(df, core, lags=[1,2,4]).dropna()
df.head()

,WMO_ID,RAINFALL_LAST_MM,TEMP_DEWPOINT_C_TDTDTD,TEMP_DRYBULB_C_TTTTTT,TEMP_WETBULB_C,WIND_SPEED_FF,RELATIVE_HUMIDITY_PC,PRESSURE_QFF_MB_DERIVED,PRESSURE_QFE_MB_DERIVED,month,...,TEMP_WETBULB_C_lag4,WIND_SPEED_FF_lag1,WIND_SPEED_FF_lag2,WIND_SPEED_FF_lag4,RAINFALL_LAST_MM_lag1,RAINFALL_LAST_MM_lag2,RAINFALL_LAST_MM_lag4,RELATIVE_HUMIDITY_PC_lag1,RELATIVE_HUMIDITY_PC_lag2,RELATIVE_HUMIDITY_PC_lag4
index,,,,,,,,,,,,,,,,,,,,,
2015-01-01 12:00:00+00:00,97230,-1.000000,0.205479,-0.085714,0.029412,-0.176471,0.055156,1008.0,1007.5,-1.0,...,0.147059,0.411765,-0.764706,0.176471,-0.996899,-0.831008,-0.999845,-0.016787,0.654676,0.189448
2015-01-01 15:00:00+00:00,97230,-0.999845,0.150685,-0.238095,-0.058824,-0.176471,0.208633,1009.3,1008.8,-1.0,...,0.323529,-0.176471,0.411765,0.411765,-1.000000,-0.996899,-1.000000,0.055156,-0.016787,-0.165468
2015-01-01 18:00:00+00:00,97230,-1.000000,0.424658,-0.180952,0.176471,0.529412,0.386091,1006.9,1006.4,-1.0,...,-0.205882,-0.176471,-0.176471,-0.764706,-0.999845,-1.000000,-0.831008,0.208633,0.055156,0.654676
2015-01-01 21:00:00+00:00,97230,-1.000000,0.424658,-0.009524,0.235294,0.294118,0.160671,1006.3,1005.8,-1.0,...,0.382353,0.529412,-0.176471,0.411765,-1.000000,-0.999845,-0.996899,0.386091,0.208633,-0.016787
2015-01-02 00:00:00+00:00,97230,-0.987597,0.315068,-0.161905,0.088235,0.882353,0.251799,1009.0,1008.5,-1.0,...,0.029412,0.294118,0.529412,-0.176471,-1.000000,-1.000000,-1.000000,0.160671,0.386091,0.055156


In [5]:

# chronological 80 / 20 split
split = int(len(df)*0.8)
train, test = df.iloc[:split], df.iloc[split:]

y_tr, y_te = train["PRESSURE_QFE_MB_DERIVED"].values.astype(np.float32), test["PRESSURE_QFE_MB_DERIVED"].values.astype(np.float32)
X_cols     = train.drop(columns=["PRESSURE_QFF_MB_DERIVED","PRESSURE_QFE_MB_DERIVED", "WMO_ID", "month"]).columns
print("Column for X: ", X_cols, len(X_cols))
X_tr       = train[X_cols].values.astype(np.float32)
X_te       = test[X_cols].values.astype(np.float32)


Column for X:  Index(['RAINFALL_LAST_MM', 'TEMP_DEWPOINT_C_TDTDTD', 'TEMP_DRYBULB_C_TTTTTT',
       'TEMP_WETBULB_C', 'WIND_SPEED_FF', 'RELATIVE_HUMIDITY_PC', 'sin_hour',
       'cos_hour', 'sin_doy', 'cos_doy', 'PRESSURE_QFF_MB_DERIVED_lag1',
       'PRESSURE_QFF_MB_DERIVED_lag2', 'PRESSURE_QFF_MB_DERIVED_lag4',
       'PRESSURE_QFE_MB_DERIVED_lag1', 'PRESSURE_QFE_MB_DERIVED_lag2',
       'PRESSURE_QFE_MB_DERIVED_lag4', 'TEMP_DEWPOINT_C_TDTDTD_lag1',
       'TEMP_DEWPOINT_C_TDTDTD_lag2', 'TEMP_DEWPOINT_C_TDTDTD_lag4',
       'TEMP_DRYBULB_C_TTTTTT_lag1', 'TEMP_DRYBULB_C_TTTTTT_lag2',
       'TEMP_DRYBULB_C_TTTTTT_lag4', 'TEMP_WETBULB_C_lag1',
       'TEMP_WETBULB_C_lag2', 'TEMP_WETBULB_C_lag4', 'WIND_SPEED_FF_lag1',
       'WIND_SPEED_FF_lag2', 'WIND_SPEED_FF_lag4', 'RAINFALL_LAST_MM_lag1',
       'RAINFALL_LAST_MM_lag2', 'RAINFALL_LAST_MM_lag4',
       'RELATIVE_HUMIDITY_PC_lag1', 'RELATIVE_HUMIDITY_PC_lag2',
       'RELATIVE_HUMIDITY_PC_lag4'],
      dtype='object') 34


In [6]:
def rmse(y_true, y_pred):
    """
    Returns root-mean-squared-error regardless of sklearn
    version (handles absence of 'squared' kwarg).
    """
    try:
        return mean_squared_error(y_true, y_pred, squared=False)
    except TypeError:
        return np.sqrt(mean_squared_error(y_true, y_pred))

In [7]:
sel_cols = [
    'PRESSURE_QFE_MB_DERIVED_lag2',
    'PRESSURE_QFE_MB_DERIVED_lag1',
    'TEMP_WETBULB_C_lag4',
    'TEMP_DRYBULB_C_TTTTTT_lag1',
    'TEMP_WETBULB_C',
    'RELATIVE_HUMIDITY_PC_lag1',
    'TEMP_DRYBULB_C_TTTTTT_lag4',
    'RAINFALL_LAST_MM_lag2'
]

In [8]:
# ──────────────────────────────────────────────────────────────
#  Grid search: n_centres × gamma × ridge  → picks lowest RMSE
# ──────────────────────────────────────────────────────────────
import itertools, pandas as pd, numpy as np
from sklearn.metrics import mean_absolute_error

# --- Editable search space ------------------------------------
grid_n   = [1900, 1950, 2000]          # hidden centres
grid_gam = [0.0001, 0.00001]        # RBF spread
grid_reg = [1e-3, 1e-2, 1e-1, 1e-4]       # ridge λ
# --------------------------------------------------------------

results = []

for n, g, r in itertools.product(grid_n, grid_gam, grid_reg):
    model = RBFNetwork(n_centres=n, gamma=g, ridge=r).fit(
                train[sel_cols].values.astype(np.float32),
                y_tr.astype(np.float32))
    pred = model.predict(test[sel_cols].values.astype(np.float32))
    res  = {
        "n_centres": n,
        "gamma":     g,
        "ridge":     r,
        "RMSE":      rmse(y_te, pred),
        "MAE":       mean_absolute_error(y_te, pred),
        "R2": r2_score(y_te, pred),
        "KGE":       kge(y_te, pred)
    }
    results.append(res)

grid_df = pd.DataFrame(results)\
            .sort_values("RMSE")\
            .reset_index(drop=True)

display(grid_df.head(10).style.format({"RMSE":"{:.3f}",
                                       "MAE":"{:.3f}",
                                       "R2": "{:.3f}",
                                       "KGE":"{:.3f}"}))
best = grid_df.iloc[0]
print(f"Results for Station {STATION_ID}\n =========")
print(f"\n🏆  Best config → n={best.n_centres}, γ={best.gamma}, "
      f"ridge={best.ridge}  •  RMSE={best.RMSE:.3f} hPa | R²={best.R2:.3f}")


,n_centres,gamma,ridge,RMSE,MAE,R2,KGE
0,1950,0.000010,0.010000,1.355,1.130,0.633,0.634
1,1900,0.000010,0.000100,1.394,1.195,0.612,0.646
2,2000,0.000010,0.100000,1.437,1.260,0.587,0.685
3,1900,0.000010,0.010000,1.444,1.246,0.583,0.616
4,1900,0.000010,0.100000,1.483,1.298,0.561,0.674
5,2000,0.000010,0.000100,1.495,1.295,0.553,0.594
6,1900,0.000010,0.001000,1.496,1.279,0.553,0.559
7,2000,0.000010,0.010000,1.527,1.321,0.534,0.582
8,1950,0.000100,0.010000,1.603,1.269,0.487,0.719
9,2000,0.000100,0.100000,1.620,1.287,0.475,0.705


Results for Station 97230

🏆  Best config → n=1950.0, γ=1e-05, ridge=0.01  •  RMSE=1.355 hPa | R²=0.633


In [9]:
# Persistence forecast: predict the previous value
y_pred_persist = y_te[:-1]          # forecast for t is value at t-1
y_true_shifted = y_te[1:]           # matching truth (start at t)

rmse_persist = rmse(y_true_shifted, y_pred_persist)
print(f"Persistence RMSE ≈ {rmse_persist:.3f} hPa")


Persistence RMSE ≈ 1.413 hPa
